In [231]:
# VerveStacks Solar Interactive Map - Enhanced Version
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up paths
data_path = Path('../data/REZoning')
solar_file = data_path / 'REZoning_Solar_atlite_cf.csv'
costs_file = data_path / 'REZoning_costs_per_kw.csv'

print("VerveStacks Solar Interactive Map - Enhanced Version")
print("=" * 60)


VerveStacks Solar Interactive Map - Enhanced Version


In [232]:
# Load data
print("Loading solar data...")
solar_data = pd.read_csv(solar_file)
costs_data = pd.read_csv(costs_file)

print(f"Solar data shape: {solar_data.shape}")
print(f"Available ISOs: {len(solar_data['ISO'].dropna().unique())} countries")
print(f"Sample ISOs: {sorted(solar_data['ISO'].dropna().unique())[:10]}")


Loading solar data...
Solar data shape: (51409, 13)
Available ISOs: 170 countries
Sample ISOs: ['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS', 'AUT', 'AZE']


In [233]:
# Core function to create solar grid data
def create_solar_grid_for_iso(iso_code, solar_df, costs_df, min_capacity_mw=1.0):
    """Create solar grid analysis for a specific ISO code."""
    
    # Filter data for the specific ISO
    iso_solar = solar_df[solar_df['ISO'] == iso_code].copy()
    
    if iso_solar.empty:
        print(f"No solar data found for ISO: {iso_code}")
        return None
    
    # Get cost data for this ISO
    iso_costs = costs_df[costs_df['iso'] == iso_code]
    
    # Clean and process the data
    iso_solar = iso_solar.dropna(subset=['lat', 'long', 'Capacity Factor'])
    iso_solar = iso_solar[iso_solar['Installed Capacity Potential (MW)'] >= min_capacity_mw]
    
    # Calculate additional metrics
    iso_solar['Total_Generation_GWh'] = (
        iso_solar['Installed Capacity Potential (MW)'] * 
        iso_solar['Capacity Factor'] * 8760 / 1000
    )
    
    # Create grid statistics
    grid_stats = {
        'iso': iso_code,
        'total_cells': len(iso_solar),
        'total_capacity_mw': iso_solar['Installed Capacity Potential (MW)'].sum(),
        'total_generation_gwh': iso_solar['Total_Generation_GWh'].sum(),
        'avg_capacity_factor': iso_solar['Capacity Factor'].mean(),
        'avg_lcoe': iso_solar['LCOE (USD/MWh)'].mean(),
        'total_suitable_area_km2': iso_solar['Suitable Area (km²)'].sum(),
        'cost_data_available': not iso_costs.empty,
        'investment_cost_usd_kw': iso_costs['invcost'].iloc[0] if not iso_costs.empty else None,
        'fixed_om_usd_kw': iso_costs['fixom'].iloc[0] if not iso_costs.empty else None,
        'data_source': 'Atlite (High-resolution ERA5 weather data)',
        'capacity_factor_quality': 'High-resolution, technology-specific modeling'
    }
    
    return {
        'grid_data': iso_solar,
        'statistics': grid_stats,
        'costs': iso_costs
    }

# Create grid for USA
sample_iso = 'USA'
print(f"Creating solar grid for ISO: {sample_iso}")
grid_result = create_solar_grid_for_iso(sample_iso, solar_data, costs_data)

if grid_result:
    stats = grid_result['statistics']
    print(f"Grid Statistics for {sample_iso}:")
    print(f"Total grid cells: {stats['total_cells']:,}")
    print(f"Total capacity: {stats['total_capacity_mw']:,.1f} MW")
    print(f"Average capacity factor: {stats['avg_capacity_factor']:.3f}")


Creating solar grid for ISO: USA
Grid Statistics for USA:
Total grid cells: 3,965
Total capacity: 8,812,713.8 MW
Average capacity factor: 0.226


In [ ]:
# ENHANCED INTERACTIVE MAP WITH IMPROVED COLOR SCHEME
import folium
from folium import plugins
import numpy as np
import pandas as pd

def create_enhanced_solar_interactive_map(grid_result, analysis_level='comprehensive'):
    """
    Create an enhanced interactive solar map with improved color scheme for better visibility.
    
    Parameters:
    -----------
    grid_result : dict
        Result from create_solar_grid_for_iso function
    analysis_level : str
        Level of analysis ('basic', 'intermediate', 'comprehensive')
    """
    
    if not grid_result:
        print("No grid data to visualize")
        return
    
    grid_data = grid_result['grid_data']
    stats = grid_result['statistics']
    
    # Calculate center point for map
    center_lat = grid_data['lat'].mean()
    center_lon = grid_data['long'].mean()
    
    # Create the enhanced map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=6,
        tiles='OpenStreetMap'
    )
    
    # Add multiple tile layers for better analysis (with proper attributions)
    folium.TileLayer('CartoDB positron', name='CartoDB Positron').add_to(m)
    folium.TileLayer('CartoDB dark_matter', name='CartoDB Dark').add_to(m)
    folium.TileLayer(
        tiles='https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}{r}.png',
        attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.',
        name='Terrain',
        overlay=False,
        control=True
    ).add_to(m)
    
    # IMPROVED color scheme with better contrast and visibility
    def get_enhanced_capacity_factor_color(cf):
        if cf >= 0.30:
            return '#006400'  # Dark Green - Excellent (very dark green for high contrast)
        elif cf >= 0.25:
            return '#228B22'  # Forest Green - High (medium green)
        elif cf >= 0.22:
            return '#FFD700'  # Gold - Good (bright gold for visibility)
        elif cf >= 0.20:
            return '#FF8C00'  # Dark Orange - Fair (vibrant orange)
        elif cf >= 0.18:
            return '#FF4500'  # Orange Red - Poor (bright red-orange)
        else:
            return '#DC143C'  # Crimson - Very Poor (deep red)
    
    # Enhanced marker size calculation
    def get_enhanced_marker_size(capacity_mw):
        if capacity_mw >= 5000:
            return 22
        elif capacity_mw >= 2000:
            return 18
        elif capacity_mw >= 1000:
            return 14
        elif capacity_mw >= 500:
            return 12
        elif capacity_mw >= 100:
            return 10
        elif capacity_mw >= 50:
            return 8
        else:
            return 6
    
    # Sample zones for performance (show max 3000 zones for comprehensive analysis)
    max_zones = 3000 if analysis_level == 'comprehensive' else 2000
    map_sample = grid_data.sample(n=min(max_zones, len(grid_data)), random_state=42).copy()
    
    print(f"Creating enhanced solar map with {len(map_sample)} zones ({analysis_level} analysis)...")
    
    # Add solar zones with enhanced analysis
    for idx, zone in map_sample.iterrows():
        # Get zone details
        capacity_factor = zone['Capacity Factor']
        capacity_mw = zone['Installed Capacity Potential (MW)']
        lcoe = zone['LCOE (USD/MWh)']
        generation_gwh = zone['Total_Generation_GWh']
        area_km2 = zone['Suitable Area (km²)']
        zone_score = zone['Zone Score']
        
        # Enhanced popup content with analysis
        popup_content = f"""
        <div style="font-family: Arial, sans-serif; width: 320px;">
            <h4 style="margin: 0; color: #2C3E50; background: #ECF0F1; padding: 8px; border-radius: 5px;">
                Solar Zone: {zone['grid_cell']}
            </h4>
            <div style="padding: 10px;">
                <div style="background: #F8F9FA; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #34495E;">📊 Resource Quality</h5>
                    <p style="margin: 2px 0;"><b>Capacity Factor:</b> {capacity_factor:.3f}</p>
                    <p style="margin: 2px 0;"><b>Zone Score:</b> {zone_score:.3f}</p>
                </div>
                
                <div style="background: #E8F5E8; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #27AE60;">⚡ Energy Potential</h5>
                    <p style="margin: 2px 0;"><b>Installed Capacity:</b> {capacity_mw:.1f} MW</p>
                    <p style="margin: 2px 0;"><b>Annual Generation:</b> {generation_gwh:.1f} GWh</p>
                    <p style="margin: 2px 0;"><b>Generation Density:</b> {generation_gwh/area_km2:.2f} GWh/km²</p>
                </div>
                
                <div style="background: #FFF3CD; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #856404;">💰 Economics</h5>
                    <p style="margin: 2px 0;"><b>LCOE:</b> ${lcoe:.2f}/MWh</p>
                    <p style="margin: 2px 0;"><b>Suitable Area:</b> {area_km2:.1f} km²</p>
                </div>
                
                <div style="background: #D1ECF1; padding: 8px; border-radius: 3px; margin: 5px 0;">
                    <h5 style="margin: 0; color: #0C5460;">📍 Location</h5>
                    <p style="margin: 2px 0;"><b>Coordinates:</b> {zone['lat']:.3f}, {zone['long']:.3f}</p>
                </div>
            </div>
        </div>
        """
        
        # Add marker with enhanced styling and CSS class
        folium.CircleMarker(
            location=[zone['lat'], zone['long']],
            radius=get_enhanced_marker_size(capacity_mw),
            popup=folium.Popup(popup_content, max_width=350),
            tooltip=f"Zone {zone['grid_cell']} (CF: {capacity_factor:.3f}, {capacity_mw:.1f} MW)",
            color='white',
            weight=3,  # Thicker white border for better contrast
            fillColor=get_enhanced_capacity_factor_color(capacity_factor),
            fillOpacity=0.85,  # Slightly more opaque for better visibility
            className='solar-circle'  # Apply CSS class for stroke-width styling
        ).add_to(m)
    
    # Enhanced legend with improved color scheme and CSS styling
    legend_html = f'''
    <style>
    .solar-circle {{
        stroke-width: 1px !important;
        stroke: white !important;
    }}
    </style>
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 300px; height: 300px; 
                background-color: white; border:3px solid #34495E; z-index:9999; 
                font-size:13px; padding: 15px; border-radius: 10px; box-shadow: 0 6px 12px rgba(0,0,0,0.15);">
    <h4 style="margin: 0 0 15px 0; color: #2C3E50; text-align: center; font-weight: bold;">Solar Resource Analysis</h4>
    
    <div style="margin-bottom: 12px;">
        <h5 style="margin: 0 0 8px 0; color: #34495E; font-weight: bold;">📊 Resource Quality (CF)</h5>
        <p style="margin: 3px 0;"><span style="color:#006400; font-weight: bold;">●</span> Excellent (≥0.30)</p>
        <p style="margin: 3px 0;"><span style="color:#228B22; font-weight: bold;">●</span> High (0.25-0.30)</p>
        <p style="margin: 3px 0;"><span style="color:#FFD700; font-weight: bold;">●</span> Good (0.22-0.25)</p>
        <p style="margin: 3px 0;"><span style="color:#FF8C00; font-weight: bold;">●</span> Fair (0.20-0.22)</p>
        <p style="margin: 3px 0;"><span style="color:#FF4500; font-weight: bold;">●</span> Poor (0.18-0.20)</p>
        <p style="margin: 3px 0;"><span style="color:#DC143C; font-weight: bold;">●</span> Very Poor (<0.18)</p>
    </div>
    
    <div style="margin-bottom: 12px;">
        <h5 style="margin: 0 0 8px 0; color: #34495E; font-weight: bold;">⚡ Zone Size (MW)</h5>
        <p style="margin: 3px 0;">● Small: <100 MW</p>
        <p style="margin: 3px 0;">●● Medium: 100-1000 MW</p>
        <p style="margin: 3px 0;">●●● Large: 1000-5000 MW</p>
        <p style="margin: 3px 0;">●●●● Very Large: >5000 MW</p>
    </div>
    
    <div style="background: #F8F9FA; padding: 10px; border-radius: 5px; border-left: 4px solid #3498DB;">
        <p style="margin: 2px 0; font-size: 11px; color: #7F8C8D; font-style: italic;"><b>Atlite ERA5 Weather Data</b></p>
        <p style="margin: 2px 0; font-size: 11px; color: #7F8C8D; font-style: italic;">Analysis Level: {analysis_level.title()}</p>
    </div>
    </div>
    '''
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Add measurement tools
    from folium.plugins import MeasureControl
    m.add_child(MeasureControl())
    
    # Add fullscreen button
    from folium.plugins import Fullscreen
    Fullscreen().add_to(m)
    
    return m

# Create enhanced interactive map with improved colors
if grid_result:
    print("Creating enhanced interactive solar map with improved color scheme...")
    enhanced_map = create_enhanced_solar_interactive_map(grid_result, 'comprehensive')
    
    # Save the enhanced map
    enhanced_map_file = f'output/enhanced_solar_zones_map_{stats["iso"]}_atlite.html'
    enhanced_map.save(enhanced_map_file)
    
    print(f"✅ Enhanced interactive map created with improved colors!")
    print(f"✅ Map saved to: {enhanced_map_file}")
    print(f"🗺️  Open the HTML file in your browser to view the enhanced interactive map!")
    
    # Display the map
    enhanced_map


Creating enhanced interactive solar map with improved color scheme...
Creating enhanced solar map with 3000 zones (comprehensive analysis)...
✅ Enhanced interactive map created with improved colors!
✅ Map saved to: output/enhanced_solar_zones_map_USA_atlite.html
🗺️  Open the HTML file in your browser to view the enhanced interactive map!


: 